In [3]:
#!pip install -U aeon
#!pip install aeon[all_extras]
import warnings
import pandas as pd
from matplotlib import pyplot as plt
from aeon.visualisation import plot_series
from sklearn.metrics import mean_absolute_percentage_error as mape
from all_functions import *
import os
from aeon.clustering.averaging import elastic_barycenter_average
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.seasonal import seasonal_decompose
from VotingCombination import VotingCombination
from sklearn.svm import SVR
import os
from tslearn.barycenters import \
    euclidean_barycenter, \
    dtw_barycenter_averaging, \
    dtw_barycenter_averaging_subgradient, \
    softdtw_barycenter
warnings.filterwarnings("ignore")
%matplotlib inline

def get_predictions_csv(path, format, start_index):
    df = pd.read_csv(path, sep=";")
    results = {}
    params = {}
    # for format in formats:
    filtered_df = df[df['DATA'] == format]
    
    columns_p1_to_p12 = filtered_df.loc[:, 'P1':'P12']
    
    # values_list = columns_p1_to_p12.values.tolist()
    values_list = columns_p1_to_p12.values.flatten().tolist()     
    results = pd.Series(values_list, index=start_index)
    params = filtered_df['PARAMS'].iloc[0]
            
    return results, params

def get_params_model(caminho_arquivo, transformation):
    df = pd.read_csv(caminho_arquivo, sep=';')
    
    df_filtrado = df[df['DATA'] == transformation]
    params_dict = ast.literal_eval(df_filtrado['PARAMS'].iloc[0])
    
    return params_dict

def convert_to_list(series_str):
    return eval(series_str)

/home/anp/anaconda3/lib/python3.11/site-packages/aeon/base/__init__.py:24: FutureWarning: The aeon package will soon be releasing v1.0.0 with the removal of legacy modules and interfaces such as BaseTransformer and BaseForecaster. This will contain breaking changes. See aeon-toolkit.org for more information. Set aeon.AEON_DEPRECATION_WARNING or the AEON_DEPRECATION_WARNING environmental variable to 'False' to disable this warning.
  warnings.warn(
/home/anp/anaconda3/lib/python3.11/site-packages/rapids_dask_dependency/dask_loader.py:36: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  mod = importlib.import_module(spec.name)


In [6]:
dirs = [
    # '../datasets/venda/mensal/uf/gasolinac/',
    # '../datasets/venda/mensal/uf/etanolhidratado/',
    # '../datasets/venda/mensal/uf/gasolinadeaviacao/',
    # '../datasets/venda/mensal/uf/glp/',
    # '../datasets/venda/mensal/uf/oleocombustivel/',
    '../datasets/venda/mensal/uf/oleodiesel/',
    '../datasets/venda/mensal/uf/querosenedeaviacao/',
    # '../datasets/venda/mensal/uf/queroseneiluminante/',
]
horizon = 12
window = 12

cols =  ['DATA', 'transformation', 'UF', 'PRODUCT', 'MODEL', 'PARAMS', 'WINDOW', 'HORIZON', 'RMSE', 'MAPE', 'POCID', 'PBE', 'MASE',
           'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12','MCPM', 
           ]
results_file = f'./combination/arima_svr_error'
transformations = ['normal', 'log', 'deseasonal']
for directory in dirs:
  path_derivado = f'{results_file}/{derivado}'
  os.makedirs(path_derivado, exist_ok=True)
  for file in os.listdir(directory):
    for transform in transformations:
      estado = file.split("_")[1].upper()
      derivado = file.split("_")[2].split(".")[0]
      series = read_series(f'../datasets/venda/mensal/uf/{derivado}/mensal_{estado.lower()}_{derivado}.csv')
      _, test = train_test_stats(series, horizon)

      dado = pd.read_csv(f'./results_hybrid/arima/{derivado}/{transform}/transform_{estado}.csv', sep=";")
      filter_arima = dado[dado['test_range'] == '2023-03_2024-02']
      columns_p1_to_p12 = filter_arima.loc[:, 'P1':'P12']
      arima_preds = columns_p1_to_p12.values.flatten().tolist()        
      dado['error_series'] = dado['error_series'].apply(convert_to_list)
      reversed_lists = dado['error_series'].iloc[::-1]

      period_index = pd.period_range(start='1993-03', end='2024-02', freq='M')
      error = []
      for lst in reversed_lists:
          error.extend(lst)
      error_series = pd.Series(error, index=period_index)
      train_error, test_error = train_test_stats(error_series, horizon)
      
      transformer = STLTransformer(sp=12)  
      result = transformer.fit(train_error)
      train_error_st = result.trend_ + result.seasonal_

      csv_path = f'{path_derivado}/transform_{estado}.csv'
      if not os.path.exists(csv_path):
        pd.DataFrame(columns=cols).to_csv(csv_path, sep=';', index=False)

      # previsao do erro direto
      data = rolling_window(error_series, window)
      data = data.dropna()
      X_train, X_test, y_train, _ = train_test_split(data, horizon)

      svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
      svr_rbf.fit(X_train, y_train)

      predictions = recursive_multistep_forecasting(X_test, svr_rbf, horizon)
      mean_norm, std_norm = get_stats_norm(error_series, horizon, window)

      predictions_rescaled = znorm_reverse(pd.Series(predictions, index=test_error.index), mean_norm, std_norm)
      preds = pd.Series(predictions_rescaled, index=test_error.index)
      combined = pd.Series(arima_preds + preds.values, index=test.index)

      y_baseline = series[-horizon*2:-horizon].values
      rmse_result = rmse(test, combined)
      mape_result = mape(test, combined)
      pocid_result = pocid(test, combined)
      pbe_result = pbe(test, combined)
      mase_result =  mase(test, combined, y_baseline)
      mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

      df_temp = pd.DataFrame({'DATA': 'error', 'transformation':  transform, 'UF': estado, 'PRODUCT': derivado, 'MODEL': 'ARIMA+SVR', 'PARAMS': 'sum', 'WINDOW': window, 'HORIZON': horizon,  
                                              'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result, 'MCPM': mcpm_result,
                                              'P1': combined[0], 'P2': combined[1], 'P3': combined[2], 'P4': combined[3], 'P5': combined[4],
                                              'P6': combined[5], 'P7': combined[6], 'P8': combined[7], 'P9': combined[8], 'P10': combined[9],
                                              'P11': combined[10], 'P12': combined[11]
                                              }, index=[0])
      df_temp.to_csv(csv_path, sep=';', mode='a', header=False, index=False)


    #previsao do erro sem componente de residual
    series_error_st = pd.concat([train_error_st, pd.Series([0,0,0,0,0,0,0,0,0,0,0,0], index=test.index)])
    data = rolling_window(series_error_st, window)

    data = data.dropna()
    X_train, X_test, y_train, _ = train_test_split(data, horizon)

    svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)
    svr_rbf.fit(X_train, y_train)

    predictions = recursive_multistep_forecasting(X_test, svr_rbf, horizon)
    mean_norm, std_norm = get_stats_norm(series_error_st, horizon, window)

    # Reescala as predições
    predictions_rescaled = znorm_reverse(pd.Series(predictions, index=test_error.index), mean_norm, std_norm)


    preds_noresid = pd.Series(predictions_rescaled, index=test_error.index)

    combined_error_noresid = pd.Series(arima_preds + preds_noresid.values, index=test.index)

    y_baseline = series[-horizon*2:-horizon].values
    rmse_result = rmse(test, combined_error_noresid)
    mape_result = mape(test, combined_error_noresid)
    pocid_result = pocid(test, combined_error_noresid)
    pbe_result = pbe(test, combined_error_noresid)
    mase_result =  mase(test, combined_error_noresid, y_baseline)
    mcpm_result = mcpm(rmse_result, mape_result, pocid_result)

    df_temp = pd.DataFrame({'DATA': 'error_noresid', 'transformation':  transform, 'UF': estado, 'PRODUCT': derivado, 'MODEL': 'ARIMA+SVR', 'PARAMS': 'sum', 'WINDOW': window, 'HORIZON': horizon,  
                                            'RMSE': rmse_result, 'MAPE': mape_result, 'POCID': pocid_result, 'PBE': pbe_result, 'MASE': mase_result, 'MCPM': mcpm_result,
                                            'P1': combined_error_noresid[0], 'P2': combined_error_noresid[1], 'P3': combined_error_noresid[2], 'P4': combined_error_noresid[3], 'P5': combined_error_noresid[4],
                                            'P6': combined_error_noresid[5], 'P7': combined_error_noresid[6], 'P8': combined_error_noresid[7], 'P9': combined_error_noresid[8], 'P10': combined_error_noresid[9],
                                            'P11': combined_error_noresid[10], 'P12': combined_error_noresid[11]
                                            }, index=[0])
    df_temp.to_csv(csv_path, sep=';', mode='a', header=False, index=False)

oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleocombustivel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
oleodiesel
